In [1]:
import pandas as pd

In [2]:
cat_school_raw = pd.read_csv("base_de_dados/cat_school_data.csv", sep=";")
cat_student_raw = pd.read_csv("base_de_dados/cat_student_data.csv", sep=";")
num_school_raw = pd.read_csv("base_de_dados/num_school_data.csv", sep=";")
num_student_raw = pd.read_csv("base_de_dados/num_student_data.csv", sep=";")

## cat_school

In [3]:
cat_school_raw

,school,variable,value
0,1,schgend,mixed
1,2,schgend,girls
2,3,schgend,mixed
3,4,schgend,mixed
4,5,schgend,mixed
...,...,...,...
125,61,type,Mxd
126,62,type,Mxd
127,63,type,Mxd
128,64,type,Sngl


In [4]:
print(cat_school_raw["school"].nunique())
print(cat_school_raw["variable"].unique())
print(cat_school_raw["value"].unique())

#  Sao 65 escolas
#  As variaveis sao ['schgend' 'type']

65
['schgend' 'type']
['mixed' 'girls' 'boys' 'Mxd' 'Sngl']


In [5]:
cat_school_piv = cat_school_raw.pivot(index='school', columns='variable', values='value').reset_index()
cat_school_piv.columns.name = None
cat_school_piv


,school,schgend,type
0,1,mixed,Mxd
1,2,girls,Sngl
2,3,mixed,Mxd
3,4,mixed,Mxd
4,5,mixed,Mxd
...,...,...,...
60,61,mixed,Mxd
61,62,mixed,Mxd
62,63,mixed,Mxd
63,64,boys,Sngl


In [6]:
print(cat_school_piv["schgend"].unique())
print(cat_school_piv["type"].unique())


['mixed' 'girls' 'boys']
['Mxd' 'Sngl']


- sao 65 escolas
- cada escola é classificada por schgend e por type:
    - "schgend" assume os valores ['mixed', 'girls', 'boys']
    - "type" assume os valores ['Mxd', 'Sngl']

In [7]:
print(cat_school_piv[cat_school_piv["type"]== "Mxd"]["schgend"].unique())  # Sempre que type == Mxd, schgend é igual a mixed
print(cat_school_piv[cat_school_piv["type"]== "Sngl"]["schgend"].unique())  # Sempre que type == Sngl, schgend é igual a girls ou boys


['mixed']
['girls' 'boys']


In [ ]:
cat_school_final = cat_school_piv.drop(columns="type")

# a coluna type eh redundante

cat_school_final

,school,schgend
0,1,mixed
1,2,girls
2,3,mixed
3,4,mixed
4,5,mixed
...,...,...
60,61,mixed
61,62,mixed
62,63,mixed
63,64,boys


## cat_student

In [9]:
cat_student_raw

,school,student,variable,value
1,1,143,intake,bottom 25%
2,1,145,intake,mid 50%
3,1,142,intake,top 25%
4,1,141,intake,mid 50%
5,1,138,intake,mid 50%
...,...,...,...,...
12173,65,46,vr,mid 50%
12174,65,59,vr,mid 50%
12175,65,60,vr,mid 50%
12176,65,61,vr,mid 50%


In [10]:
cat_student_single_id = cat_student_raw.copy()
cat_student_single_id["student_id"] = cat_student_raw["school"].astype(str) + "_" + cat_student_raw["student"].astype(str)
cat_student_single_id = cat_student_single_id.drop(columns=["school", "student"])
cat_student_single_id

# A nova coluna student_id eh um id unico para cada estudante, independente da escola

,variable,value,student_id
1,intake,bottom 25%,1_143
2,intake,mid 50%,1_145
3,intake,top 25%,1_142
4,intake,mid 50%,1_141
5,intake,mid 50%,1_138
...,...,...,...
12173,vr,mid 50%,65_46
12174,vr,mid 50%,65_59
12175,vr,mid 50%,65_60
12176,vr,mid 50%,65_61


In [11]:
print(cat_student_single_id["student_id"].nunique())
print(cat_student_raw["variable"].unique())
print(cat_student_raw["value"].unique())

#  No total sao 4010 estudantes
#  As variaveis sao ['intake' 'sex' 'vr']

4010
['intake' 'sex' 'vr']
['bottom 25%' 'mid 50%' 'top 25%' 'F' 'M']


In [12]:
duplicatas = cat_student_single_id[cat_student_single_id.duplicated(subset=["student_id", "variable"], keep=False)]
print(duplicatas["student_id"].unique())
duplicatas.sort_values(["student_id", "variable"])

# 4 estudantes estao duplicados, e apenas a amostra 43_86 esta com valores incoerentes de intake (iremos elimina-la)

['43_86' '50_39' '52_2' '52_21']


,variable,value,student_id
2758,intake,mid 50%,43_86
2759,intake,top 25%,43_86
6817,sex,M,43_86
6818,sex,F,43_86
10876,vr,top 25%,43_86
10877,vr,top 25%,43_86
3227,intake,mid 50%,50_39
3228,intake,mid 50%,50_39
7286,sex,F,50_39
7287,sex,F,50_39


In [13]:
cat_student_clean = cat_student_single_id.drop_duplicates(subset=["student_id", "variable"])
cat_student_clean = cat_student_clean[cat_student_clean["student_id"] != "43_86"]
print(cat_student_clean["student_id"].nunique())

#  Agora possuimos 4009 estudantes, tudo certo.

4009


In [14]:
cat_student_piv = cat_student_clean.pivot(index="student_id", columns="variable", values="value").reset_index()
cat_student_piv.columns.name = None
cat_student_piv

,student_id,intake,sex,vr
0,10_101,mid 50%,F,mid 50%
1,10_102,bottom 25%,M,mid 50%
2,10_11,bottom 25%,F,mid 50%
3,10_111,mid 50%,M,mid 50%
4,10_112,mid 50%,M,mid 50%
...,...,...,...,...
4004,9_79,top 25%,M,bottom 25%
4005,9_8,mid 50%,M,bottom 25%
4006,9_86,top 25%,NaN,bottom 25%
4007,9_9,top 25%,M,bottom 25%


In [15]:
print(cat_student_piv["intake"].unique())
print(cat_student_piv["sex"].unique())
print(cat_student_piv["vr"].unique())

#  Apareceram nan em todas as colunas
#  O genero de cada aluno especifico nao interessa, queremos saber qual o tipo da escola dele


['mid 50%' 'bottom 25%' 'top 25%' nan]
['F' 'M' nan]
['mid 50%' 'top 25%' nan 'bottom 25%']


In [16]:
cat_student_final = cat_student_piv.copy()
cat_student_final


,student_id,intake,sex,vr
0,10_101,mid 50%,F,mid 50%
1,10_102,bottom 25%,M,mid 50%
2,10_11,bottom 25%,F,mid 50%
3,10_111,mid 50%,M,mid 50%
4,10_112,mid 50%,M,mid 50%
...,...,...,...,...
4004,9_79,top 25%,M,bottom 25%
4005,9_8,mid 50%,M,bottom 25%
4006,9_86,top 25%,NaN,bottom 25%
4007,9_9,top 25%,M,bottom 25%


## num_school

In [17]:
num_school_raw

,school,variable,value
0,1,schavg,"0,1661752"
1,2,schavg,"0,3951492"
2,3,schavg,"0,5141552"
3,4,schavg,"0,0917642"
4,5,schavg,"0,2105252"
...,...,...,...
60,61,schavg,"-0,020198"
61,62,schavg,"0,1673862"
62,63,schavg,"0,1562112"
63,64,schavg,"0,4341442"


In [18]:
print(num_school_raw["variable"].unique())


['schavg']


In [19]:
num_school_final = num_school_raw.drop(columns="variable").rename(columns={"value": "schavg"})
num_school_final["schavg"] = num_school_final["schavg"].str.replace(",", ".").astype(float)
num_school_final


,school,schavg
0,1,0.166175
1,2,0.395149
2,3,0.514155
3,4,0.091764
4,5,0.210525
...,...,...
60,61,-0.020198
61,62,0.167386
62,63,0.156211
63,64,0.434144


In [20]:
len(num_school_final[num_school_final["schavg"] < 0])


38

## num_student

In [21]:
num_student_raw

,school,student,variable,value
1,1,143.0,normexam,"0,2613242"
2,1,145.0,normexam,"0,1340672"
3,1,142.0,normexam,"-1,723882"
4,1,141.0,normexam,"0,9675862"
5,1,138.0,normexam,"0,5443412"
...,...,...,...,...
8114,65,46.0,standLRT,"1,8588312"
8115,65,59.0,standLRT,"-1,03397"
8116,65,60.0,standLRT,"0,4537562"
8117,65,61.0,standLRT,"-0,786016"


In [22]:
num_student_raw.info()

#  Possuimos 90 alunos nao identificados dentro de sua escola (NaN)
#  Possuimos 101 values nao preenchidos (NaN)

#  Como o objetivo eh avaliar as escolas (nao os alunos individualmente), esses dados ainda poderao ser aproveitados


<class 'pandas.core.frame.DataFrame'>
Index: 8118 entries, 1 to 8118
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   school    8118 non-null   int64  
 1   student   8028 non-null   float64
 2   variable  8118 non-null   object 
 3   value     8017 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 317.1+ KB


In [34]:
num_student_filled = num_student_raw.copy()

# Remove o .0 ao final dos float, e depois transforma em string, mantendo os NaN
num_student_filled["student"] = num_student_filled["student"].apply(lambda x: x if pd.isna(x) else str(int(x)))  

# Substitui os NaN por naX, com X comecando em 1 e incrementando, assim NAO colocamos alunos diferentes como repetidos
counter = iter(range(1, num_student_filled["student"].isna().sum() + 1))
num_student_filled["student"] = num_student_filled["student"].apply(lambda x: f"na{next(counter)}" if pd.isna(x) else x)


num_student_filled.info()

num_student_filled

<class 'pandas.core.frame.DataFrame'>
Index: 8118 entries, 1 to 8118
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   school    8118 non-null   int64 
 1   student   8118 non-null   object
 2   variable  8118 non-null   object
 3   value     8017 non-null   object
dtypes: int64(1), object(3)
memory usage: 317.1+ KB


,school,student,variable,value
1,1,143,normexam,"0,2613242"
2,1,145,normexam,"0,1340672"
3,1,142,normexam,"-1,723882"
4,1,141,normexam,"0,9675862"
5,1,138,normexam,"0,5443412"
...,...,...,...,...
8114,65,46,standLRT,"1,8588312"
8115,65,59,standLRT,"-1,03397"
8116,65,60,standLRT,"0,4537562"
8117,65,61,standLRT,"-0,786016"


In [24]:
num_student_filled["student_id"] = num_student_filled["school"].astype(str) + "_" + num_student_filled["student"].astype(str)

num_student_single_id = num_student_filled.drop(columns=["school", "student"])

num_student_single_id

,variable,value,student_id
1,normexam,"0,2613242",1_143
2,normexam,"0,1340672",1_145
3,normexam,"-1,723882",1_142
4,normexam,"0,9675862",1_141
5,normexam,"0,5443412",1_138
...,...,...,...
8114,standLRT,"1,8588312",65_46
8115,standLRT,"-1,03397",65_59
8116,standLRT,"0,4537562",65_60
8117,standLRT,"-0,786016",65_61


In [25]:
print(num_student_single_id["variable"].unique())
print(num_student_single_id["student_id"].nunique())

# As variaveis sao ['normexam' 'standLRT']
# Aqui possuimos 4100 estudantes


['normexam' 'standLRT']
4100


In [26]:
duplicatas = num_student_single_id[num_student_single_id.duplicated(subset=["student_id", "variable"], keep=False)]
print(duplicatas["student_id"].unique())
duplicatas.sort_values(["student_id", "variable"])

# os mesmos 4 estudantes duplicados do dataframe cat_student apareceram aqui
# a amostra 43_86 foi eliminada, e as demais irei manter o promeiro valor

['43_86' '50_39' '52_2' '52_21']


,variable,value,student_id
2758,normexam,"-0,85267",43_86
2759,normexam,"0,8219882",43_86
6817,standLRT,"0,1231502",43_86
6818,standLRT,"-0,042152",43_86
3227,normexam,NaN,50_39
3228,normexam,"-2,108644",50_39
7286,standLRT,"-0,124804",50_39
7287,standLRT,"-1,199273",50_39
3314,normexam,"-0,197611",52_2
3316,normexam,"2,0397092",52_2


In [27]:
num_student_clean = num_student_single_id.drop_duplicates(subset=["student_id", "variable"])
num_student_clean = num_student_clean[num_student_clean["student_id"] != "43_86"]

print(num_student_clean["student_id"].nunique())

# 4099 estudantes

4099


In [28]:
num_student_piv = num_student_clean.pivot(index="student_id", columns="variable", values="value").reset_index()
num_student_piv.columns.name = None
num_student_piv

,student_id,normexam,standLRT
0,10_101,"-0,623051","-0,207455"
1,10_102,"-1,118624","0,6190592"
2,10_11,"0,2613242","0,9496652"
3,10_111,"-0,419801","0,2058022"
4,10_112,"0,0043222","0,0404992"
...,...,...,...
4094,9_79,"-0,492781","-1,61253"
4095,9_8,"-0,265159","-0,124804"
4096,9_86,"-0,699505","-2,19109"
4097,9_9,"-2,108644","-2,934953"


In [29]:
num_student_final = num_student_piv.copy()
num_student_final

,student_id,normexam,standLRT
0,10_101,"-0,623051","-0,207455"
1,10_102,"-1,118624","0,6190592"
2,10_11,"0,2613242","0,9496652"
3,10_111,"-0,419801","0,2058022"
4,10_112,"0,0043222","0,0404992"
...,...,...,...
4094,9_79,"-0,492781","-1,61253"
4095,9_8,"-0,265159","-0,124804"
4096,9_86,"-0,699505","-2,19109"
4097,9_9,"-2,108644","-2,934953"


## Juntando tudo

O que interessa eh o normexam de cada escola

In [30]:
cat_school_final.head()

,school,schgend
0,1,mixed
1,2,girls
2,3,mixed
3,4,mixed
4,5,mixed


In [32]:
num_school_final.head()

,school,schavg
0,1,0.166175
1,2,0.395149
2,3,0.514155
3,4,0.091764
4,5,0.210525


In [31]:
cat_student_final.head()

,student_id,intake,sex,vr
0,10_101,mid 50%,F,mid 50%
1,10_102,bottom 25%,M,mid 50%
2,10_11,bottom 25%,F,mid 50%
3,10_111,mid 50%,M,mid 50%
4,10_112,mid 50%,M,mid 50%


In [33]:
num_student_final.head()

,student_id,normexam,standLRT
0,10_101,"-0,623051","-0,207455"
1,10_102,"-1,118624","0,6190592"
2,10_11,"0,2613242","0,9496652"
3,10_111,"-0,419801","0,2058022"
4,10_112,"0,0043222","0,0404992"
